# 第 10 章 数据聚合与分组运算

In [4]:
import pandas as pd

In [5]:
import numpy as np

## 10.1 GroupBy 机制

In [6]:
df=pd.DataFrame({'key1':list('aabba')
                , 'key2':['one','two','one','two','one']
                , 'data1':np.random.randn(5)
                , 'data2':np.random.randn(5)})

In [7]:
df

,key1,key2,data1,data2
0,a,one,-0.203284,0.402536
1,a,two,-0.647550,0.423168
2,b,one,2.002774,-0.662700
3,b,two,0.383462,0.561235
4,a,one,1.644504,-1.578988


In [8]:
grouped=df['data1'].groupby(df['key1'])

In [9]:
grouped

In [10]:
grouped.mean()

key1
a    0.264556
b    1.193118
Name: data1, dtype: float64

In [11]:
[i for i in grouped]

[('a', 0   -0.203284
  1   -0.647550
  4    1.644504
  Name: data1, dtype: float64), ('b', 2    2.002774
  3    0.383462
  Name: data1, dtype: float64)]

In [12]:
means=df['data1'].groupby([df['key1'],df['key2']]).mean()

In [13]:
means

key1  key2
a     one     0.720610
      two    -0.647550
b     one     2.002774
      two     0.383462
Name: data1, dtype: float64

In [14]:
means.unstack()

key2,one,two
key1,,
a,0.720610,-0.647550
b,2.002774,0.383462


In [15]:
states=np.array(['Ohio','California','California','Ohio','Ohio'])

In [16]:
years=np.array([2005,2005,2006,2005,2006])

In [17]:
df['data1'].groupby([states,years]).mean()

California  2005   -0.647550
            2006    2.002774
Ohio        2005    0.090089
            2006    1.644504
Name: data1, dtype: float64

In [22]:
df

,key1,key2,data1,data2
0,a,one,-0.203284,0.402536
1,a,two,-0.647550,0.423168
2,b,one,2.002774,-0.662700
3,b,two,0.383462,0.561235
4,a,one,1.644504,-1.578988


In [19]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.264556,-0.251095
b,1.193118,-0.050732


In [20]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.720610 -0.588226
     two  -0.647550  0.423168
b    one   2.002774 -0.662700
     two   0.383462  0.561235

In [21]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代

In [23]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.203284  0.402536
1    a  two -0.647550  0.423168
4    a  one  1.644504 -1.578988
b
  key1 key2     data1     data2
2    b  one  2.002774 -0.662700
3    b  two  0.383462  0.561235


In [24]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.203284  0.402536
4    a  one  1.644504 -1.578988
('a', 'two')
  key1 key2    data1     data2
1    a  two -0.64755  0.423168
('b', 'one')
  key1 key2     data1   data2
2    b  one  2.002774 -0.6627
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.383462  0.561235


In [25]:
pieces=dict(list(df.groupby('key1')))

In [26]:
pieces['b']

,key1,key2,data1,data2
2,b,one,2.002774,-0.662700
3,b,two,0.383462,0.561235


In [27]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [29]:
groupby=df.groupby(df.dtypes,1)

In [30]:
for dtype,group in groupby:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.203284  0.402536
1 -0.647550  0.423168
2  2.002774 -0.662700
3  0.383462  0.561235
4  1.644504 -1.578988
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选取一列或列的子集 

In [31]:
df.groupby('key1')['data1']

In [32]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.588226
     two   0.423168
b    one  -0.662700
     two   0.561235

In [33]:
s_groupby=df.groupby(['key1','key2'])['data2']

In [34]:
s_groupby

In [35]:
s_groupby.mean()

key1  key2
a     one    -0.588226
      two     0.423168
b     one    -0.662700
      two     0.561235
Name: data2, dtype: float64

### 通过字典或Series进行分组 

In [38]:
people=pd.DataFrame(np.random.randn(5,5)
                   , columns=list('abcde')
                   , index=['Joe','Steve','Wes','Jim','Travis'])

In [39]:
people.iloc[2:3,[1,2]]=np.nan

In [40]:
people

,a,b,c,d,e
Joe,-0.742648,1.871137,0.999381,0.969662,-1.268532
Steve,-0.624493,0.138207,0.737370,0.775085,0.481656
Wes,-2.137798,NaN,NaN,-0.535474,0.961430
Jim,-0.122597,-0.500548,-1.934974,-0.866544,-2.568289
Travis,1.297726,0.206294,0.142107,0.234405,0.656585


In [41]:
mapping={'a':'red',
         'b':'red',
        'c':'blue',
        'd':'blue',
        'e':'red',
        'f':'orange'}

In [42]:
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [43]:
by_columns=people.groupby(mapping,axis=1)

In [45]:
by_columns.sum()

,blue,red
Joe,1.969043,-0.140044
Steve,1.512456,-0.004630
Wes,-0.535474,-1.176367
Jim,-2.801518,-3.191435
Travis,0.376511,2.160604


In [46]:
map_series=pd.Series(mapping)

In [47]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [48]:
people.groupby(mapping, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 通过函数进行分组 

In [49]:
people.groupby(len).sum()

,a,b,c,d,e
3,-3.003043,1.370588,-0.935594,-0.432355,-2.875392
5,-0.624493,0.138207,0.737370,0.775085,0.481656
6,1.297726,0.206294,0.142107,0.234405,0.656585


### 根据索引级别分组

In [51]:
columns=pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                  [1,3,5,1,3]],
                                  names=['cty','tenor'])

In [52]:
hier_df=pd.DataFrame(np.random.randn(4,5)
                    , columns=columns)

In [53]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.976796  1.585445 -0.519733  0.235083  0.455647
1      0.178463 -0.760334  0.317427  0.172346 -0.365387
2     -1.592667  0.072010  0.534662 -0.767560 -0.284519
3     -0.850524 -0.970281 -2.007614  0.288892 -0.213731

In [55]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 数据聚合

In [56]:
df

,key1,key2,data1,data2
0,a,one,-0.203284,0.402536
1,a,two,-0.647550,0.423168
2,b,one,2.002774,-0.662700
3,b,two,0.383462,0.561235
4,a,one,1.644504,-1.578988


In [57]:
groupby=df.groupby('key1')

In [58]:
groupby['data1'].quantile(0.9)

key1
a    1.274946
b    1.840843
Name: data1, dtype: float64

In [59]:
def peak_to_peak(arr):
    return arr.max()-arr.min()

In [60]:
grouped.agg(peak_to_peak)

key1
a    2.292054
b    1.619312
Name: data1, dtype: float64

In [61]:
grouped.describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,0.264556,1.215538,-0.647550,-0.425417,-0.203284,0.720610,1.644504
b,2.0,1.193118,1.145026,0.383462,0.788290,1.193118,1.597946,2.002774


### 面向列的多函数应用

In [62]:
#  没有数据

## 10.3 apply：一般性的拆分-应用-合并

### 分位数和痛分析

In [63]:
frame=pd.DataFrame({'data1':np.random.randn(1000)
                   , 'data2':np.random.randn(1000)})

In [64]:
quartiles=pd.cut(frame.data1,4)

In [65]:
quartiles[:10]

0    (-1.802, -0.12]
1     (-0.12, 1.561]
2     (-0.12, 1.561]
3    (-1.802, -0.12]
4     (-0.12, 1.561]
5     (-0.12, 1.561]
6     (-0.12, 1.561]
7    (-1.802, -0.12]
8    (-1.802, -0.12]
9    (-1.802, -0.12]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.49, -1.802] < (-1.802, -0.12] < (-0.12, 1.561] < (1.561, 3.242]]

In [66]:
def get_stats(group):
    return {'min':group.min()
           ,'max':group.max()
           ,'count':group.count()
           ,'mean':group.mean()}

In [67]:
grouped=frame.data2.groupby(quartiles)

In [68]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.49, -1.802]",35.0,1.655456,-0.222523,-2.086319
"(-1.802, -0.12]",428.0,3.660525,0.102066,-2.362876
"(-0.12, 1.561]",489.0,2.978770,0.000915,-2.852596
"(1.561, 3.242]",48.0,1.850796,-0.110206,-2.250795


In [69]:
grouping=pd.qcut(frame.data1,10,labels=False)

In [70]:
grouped=frame.data2.groupby(grouping)

In [71]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.794184,0.109826,-2.125896
1,100.0,2.839326,0.099728,-2.298508
2,100.0,3.660525,0.134660,-2.362876
3,100.0,3.028383,-0.062320,-2.360613
4,100.0,3.337829,0.084995,-1.731205
5,100.0,2.475779,0.051312,-2.852596
6,100.0,2.077003,0.032112,-2.531463
7,100.0,1.983208,-0.104266,-2.405272
8,100.0,1.895195,-0.013132,-2.455026


### 示例：用特定于分组的值填充缺失值

In [72]:
s=pd.Series(np.random.randn(6))

In [73]:
s[:2]=np.nan

In [74]:
s

0         NaN
1         NaN
2    0.205437
3   -1.640223
4    1.301479
5    0.577709
dtype: float64

In [75]:
states=['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']

In [76]:
group_key=['East']*4+['West']*4

In [77]:
data=pd.Series(np.random.randn(8)
              , index=states)

In [78]:
data

Ohio         -0.994527
New York      0.180409
Vermont      -0.793148
Florida      -0.348890
Oregon       -0.895568
Nevada        0.686718
California   -0.832075
Idaho         0.416427
dtype: float64

In [79]:
data[['Vermont','Nevada','Idaho']]=np.nan

In [80]:
data

Ohio         -0.994527
New York      0.180409
Vermont            NaN
Florida      -0.348890
Oregon       -0.895568
Nevada             NaN
California   -0.832075
Idaho              NaN
dtype: float64

In [81]:
fill_mean=lambda g:g.fillna(g.mean())

In [82]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.994527
New York      0.180409
Vermont      -0.387669
Florida      -0.348890
Oregon       -0.895568
Nevada       -0.863822
California   -0.832075
Idaho        -0.863822
dtype: float64

In [83]:
fill_values={'East':0.5
            , 'West':-1}

In [84]:
fill_func=lambda g:g.fillna(fill_values[g.name])

In [85]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.994527
New York      0.180409
Vermont       0.500000
Florida      -0.348890
Oregon       -0.895568
Nevada       -1.000000
California   -0.832075
Idaho        -1.000000
dtype: float64

### 示例：随机采样和排列

In [86]:
suits=list('HSCD')

In [87]:
card_val=(list(range(1,11))+[10]*3)*4

In [95]:
base_names=['A']+list(range(2,11))+list('JKQ')

In [96]:
cards=[]

In [97]:
for suit in list('HSCD'):
    cards.extend(str(num)+suit for num in base_names)

In [98]:
deck=pd.Series(card_val,index=cards)

In [99]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [100]:
def draw(deck,n=5):
    return deck.sample(n)

In [101]:
draw(deck)

5H     5
6H     6
9S     9
KC    10
QD    10
dtype: int64

In [102]:
get_suit=lambda card:card[-1]

In [103]:
deck.groupby(get_suit).apply(draw, n=2)

C  7C     7
   8C     8
D  9D     9
   6D     6
H  6H     6
   KH    10
S  KS    10
   6S     6
dtype: int64

In [104]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

4C      4
QC     10
10D    10
AD      1
KH     10
JH     10
5S      5
9S      9
dtype: int64

### 示例：分组加权平均数和相关系数

In [105]:
df=pd.DataFrame({'category':list('aaaabbbb')
                ,'data':np.random.randn(8)
                ,'weights':np.random.rand(8)})

In [106]:
grouped=df.groupby('category')

In [107]:
get_wavg=lambda g:np.average(g['data']
                            , weights=g['weights'])

In [108]:
grouped.apply(get_wavg)

category
a    0.018968
b   -0.318724
dtype: float64

### 示例：组级别的线性回归

In [109]:
import statsmodels.api as sm

In [110]:
def regress(data,yvar,xvars):
    Y=data[yvar]
    X=data[xvar]
    X['intercept']=1.
    result=sm.OLS(Y,X).fit()
    return result.params

## 10.4 透视表和交叉表